In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
train_labels = pd.read_csv('new/new_data.csv')

In [4]:
import cv2
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

In [5]:
from tqdm import tqdm

In [17]:
immatrix = []
imlabels = []
rows,cols = 224,224
img_list = os.listdir('new/compress6000')

In [18]:
for file in tqdm(img_list):
    imlabels.append(list(train_labels.loc[train_labels.image==file[:-5], :].values[0]))
    img = cv2.imread("new/compress6000/" + file)   
    img = cv2.resize(img,(rows,cols))
    immatrix.append(img)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6470/6470 [00:42<00:00, 151.73it/s]


In [26]:
np.save('compress.npy', np.asarray(immatrix))

In [28]:
from sklearn.utils import shuffle
data,Label = shuffle(immatrix,imlabels, random_state=2)
# train_data = [data,Label]

list

In [34]:
X, y = data,Label

In [24]:
# (X,y) = (train_data[0],train_data[1])

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), np.asarray(y), test_size=0.2, random_state=4)

In [40]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [43]:
X_train[:500] /= 255
X_test[:500] /= 255

In [45]:
X_train[500:1000] /= 255
X_test[500:1000] /= 255

In [46]:
X_train[1000:2000] /= 255
X_test[1000:2000] /= 255

In [47]:
X_train[2000:4000] /= 255
X_test[2000:4000] /= 255

In [48]:
X_train[4000:] /= 255
X_test[4000:] /= 255

In [ ]:
# #Normalizing Data Between [0,1]
# X_train /= 255
# X_test /= 255

# print('X_train shape:', X_train.shape)
# print(X_train.shape[0], 'train samples')
# print(X_test.shape[0], 'test samples')
    

In [61]:
y_train.T[2]

array(['2', '1', '1', ..., '2', '3', '1'], dtype='<U11')

In [63]:
y_test.T[2]

array(['0', '0', '2', ..., '1', '2', '2'], dtype='<U11')

In [64]:
from keras.utils import np_utils

n_classes = 5

Y_train = np_utils.to_categorical(y_train.T[2], n_classes)
Y_test = np_utils.to_categorical(y_test.T[2], n_classes)

In [66]:
Y_train.shape

(5176, 5)

In [67]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
print(base_model.summary())

54034432/58889256 [==========================>...] - ETA: 15:2 - ETA: 22:4 - ETA: 11:5 - ETA: 6:1 - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 59 - ETA: 59 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 56 - ETA: 56 - ETA: 59 - ETA: 56 - ETA: 57 - ETA: 56 - ETA: 56 - ETA: 57 - ETA: 55 - ETA: 57 - ETA: 57 - ETA: 55 - ETA: 56 - ETA: 54 - ETA: 55 - ETA: 54 - ETA: 55 - ETA: 53 - ETA: 53 - ETA: 53 - ETA: 53 - ETA: 53 - ETA: 52 - ETA: 52 - ETA: 53 - ETA: 52 - ETA: 52 - ETA: 53 - ETA: 51 - ETA: 52 - ETA: 51 - ETA: 51 - ETA: 51 - ETA: 51 - ETA: 50 - ETA: 50 - ETA: 50 - ETA: 50 - ETA: 50 - ETA: 50 - ETA: 50 - ETA: 49 -

In [68]:
from keras.models import Sequential
from keras.layers import Dense , Flatten , Dropout, Activation

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(5, activation='softmax'))

In [69]:
from keras.models import Model

model=Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [70]:
from keras.preprocessing.image import ImageDataGenerator

# create generators  - training data will be augmented images
validationdatagenerator = ImageDataGenerator()
traindatagenerator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1 )

batchsize=8
train_generator=traindatagenerator.flow(X_train, Y_train, batch_size=batchsize) 
validation_generator=validationdatagenerator.flow(X_test, Y_test,batch_size=batchsize)

In [71]:
for layer in model.layers[:15]:
    layer.trainable = False

In [72]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
     optimizer=optimizers.RMSprop(lr=1e-4),
     metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [73]:
batch_size =32
nb_epochs =10
nb_classes = 5

In [31]:
model.fit(x = X_train,y=Y_train)

Epoch 1/1
4424/4424 [==============================] - ETA: 1:21:27 - loss: 1.8127 - acc: 0.21 - ETA: 40:52 - loss: 1.8205 - acc: 0.2188 - ETA: 27:20 - loss: 1.8389 - acc: 0.20 - ETA: 20:33 - loss: 1.8584 - acc: 0.17 - ETA: 16:29 - loss: 1.8973 - acc: 0.18 - ETA: 13:46 - loss: 1.8678 - acc: 0.19 - ETA: 11:50 - loss: 1.8515 - acc: 0.22 - ETA: 10:22 - loss: 1.8008 - acc: 0.25 - ETA: 9:14 - loss: 1.7679 - acc: 0.2743 - ETA: 8:20 - loss: 1.7384 - acc: 0.293 - ETA: 7:35 - loss: 1.6909 - acc: 0.309 - ETA: 6:58 - loss: 1.6499 - acc: 0.333 - ETA: 6:26 - loss: 1.6382 - acc: 0.351 - ETA: 5:59 - loss: 1.5933 - acc: 0.379 - ETA: 5:35 - loss: 1.5404 - acc: 0.408 - ETA: 5:15 - loss: 1.5385 - acc: 0.419 - ETA: 4:56 - loss: 1.5001 - acc: 0.439 - ETA: 4:40 - loss: 1.4750 - acc: 0.454 - ETA: 4:26 - loss: 1.4553 - acc: 0.467 - ETA: 4:12 - loss: 1.4311 - acc: 0.482 - ETA: 4:00 - loss: 1.4102 - acc: 0.494 - ETA: 3:49 - loss: 1.3926 - acc: 0.502 - ETA: 3:39 - loss: 1.3963 - acc: 0.508 - ETA: 3:30 - loss: 1.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=int(len(X_train)/batchsize), 
                epochs=6, validation_data=validation_generator, 
                validation_steps=int(len(X_test)/batchsize))

Epoch 1/6


195/647 [========>.....................] - ETA: 3:32:33 - loss: 1.6505 - acc: 0.12 - ETA: 2:27:46 - loss: 1.5610 - acc: 0.25 - ETA: 2:05:15 - loss: 2.4658 - acc: 0.25 - ETA: 1:54:23 - loss: 2.1757 - acc: 0.31 - ETA: 1:47:31 - loss: 2.1486 - acc: 0.25 - ETA: 1:42:56 - loss: 2.0478 - acc: 0.22 - ETA: 1:39:59 - loss: 1.9840 - acc: 0.23 - ETA: 1:37:18 - loss: 1.9293 - acc: 0.26 - ETA: 1:35:17 - loss: 1.8370 - acc: 0.30 - ETA: 1:33:38 - loss: 1.8479 - acc: 0.30 - ETA: 1:32:17 - loss: 1.8360 - acc: 0.29 - ETA: 1:31:10 - loss: 1.8251 - acc: 0.29 - ETA: 1:30:10 - loss: 1.7988 - acc: 0.29 - ETA: 1:29:17 - loss: 1.7686 - acc: 0.31 - ETA: 1:28:33 - loss: 1.7590 - acc: 0.31 - ETA: 1:27:52 - loss: 1.7445 - acc: 0.32 - ETA: 1:27:13 - loss: 1.7275 - acc: 0.32 - ETA: 1:26:40 - loss: 1.7212 - acc: 0.31 - ETA: 1:26:07 - loss: 1.6999 - acc: 0.33 - ETA: 1:25:39 - loss: 1.6935 - acc: 0.33 - ETA: 1:25:11 - loss: 1.6935 - acc: 0.33 - ETA: 1:24:47 - loss: 1.6722 - acc: 0.34 - ETA: 1:24:21 - loss: 1.6580 - acc

603/647 [==========================>...] - ETA: 31:24 - loss: 1.5029 - acc: 0.38 - ETA: 31:16 - loss: 1.5036 - acc: 0.38 - ETA: 31:09 - loss: 1.5035 - acc: 0.38 - ETA: 31:01 - loss: 1.5040 - acc: 0.38 - ETA: 30:54 - loss: 1.5036 - acc: 0.38 - ETA: 30:46 - loss: 1.5042 - acc: 0.38 - ETA: 30:38 - loss: 1.5040 - acc: 0.38 - ETA: 30:30 - loss: 1.5037 - acc: 0.38 - ETA: 30:23 - loss: 1.5040 - acc: 0.38 - ETA: 30:15 - loss: 1.5044 - acc: 0.38 - ETA: 30:07 - loss: 1.5043 - acc: 0.38 - ETA: 30:00 - loss: 1.5038 - acc: 0.38 - ETA: 29:52 - loss: 1.5027 - acc: 0.38 - ETA: 29:44 - loss: 1.5026 - acc: 0.38 - ETA: 29:37 - loss: 1.5031 - acc: 0.38 - ETA: 29:29 - loss: 1.5025 - acc: 0.38 - ETA: 29:21 - loss: 1.5032 - acc: 0.38 - ETA: 29:14 - loss: 1.5026 - acc: 0.38 - ETA: 29:06 - loss: 1.5037 - acc: 0.38 - ETA: 28:58 - loss: 1.5043 - acc: 0.38 - ETA: 28:51 - loss: 1.5041 - acc: 0.38 - ETA: 28:43 - loss: 1.5035 - acc: 0.38 - ETA: 28:35 - loss: 1.5030 - acc: 0.38 - ETA: 28:28 - loss: 1.5028 - acc: 0.38

646/647 [============================>.] - ETA: 5:26 - loss: 1.4921 - acc: 0.396 - ETA: 5:19 - loss: 1.4918 - acc: 0.396 - ETA: 5:11 - loss: 1.4916 - acc: 0.396 - ETA: 5:03 - loss: 1.4915 - acc: 0.396 - ETA: 4:56 - loss: 1.4921 - acc: 0.396 - ETA: 4:48 - loss: 1.4919 - acc: 0.396 - ETA: 4:41 - loss: 1.4916 - acc: 0.396 - ETA: 4:33 - loss: 1.4916 - acc: 0.396 - ETA: 4:25 - loss: 1.4913 - acc: 0.396 - ETA: 4:18 - loss: 1.4910 - acc: 0.396 - ETA: 4:10 - loss: 1.4909 - acc: 0.396 - ETA: 4:03 - loss: 1.4907 - acc: 0.397 - ETA: 3:55 - loss: 1.4907 - acc: 0.397 - ETA: 3:47 - loss: 1.4907 - acc: 0.396 - ETA: 3:40 - loss: 1.4908 - acc: 0.396 - ETA: 3:32 - loss: 1.4907 - acc: 0.396 - ETA: 3:25 - loss: 1.4907 - acc: 0.396 - ETA: 3:17 - loss: 1.4903 - acc: 0.396 - ETA: 3:09 - loss: 1.4900 - acc: 0.396 - ETA: 3:02 - loss: 1.4899 - acc: 0.396 - ETA: 2:54 - loss: 1.4901 - acc: 0.396 - ETA: 2:47 - loss: 1.4900 - acc: 0.396 - ETA: 2:39 - loss: 1.4900 - acc: 0.396 - ETA: 2:31 - loss: 1.4903 - acc: 0.395

In [38]:
# serialize model to JSON
model_json = model.to_json()
with open("model_train_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("DR_train_1.h5")
print("Saved model to disk")

Saved model to disk


In [39]:
with tf.device("/gpu:0"):
    probs = model.predict(X_test)

In [40]:
probs[:10]

array([[0.80414534, 0.05421184, 0.12046093, 0.01211276, 0.00906908],
       [0.70082736, 0.07930345, 0.17942877, 0.02229862, 0.01814172],
       [0.68425024, 0.08667489, 0.18328655, 0.02499642, 0.02079195],
       [0.7315191 , 0.07174817, 0.16315447, 0.01880349, 0.01477475],
       [0.7310789 , 0.07118486, 0.1679979 , 0.0164353 , 0.01330303],
       [0.7377781 , 0.06884215, 0.16138896, 0.01807972, 0.01391109],
       [0.6926523 , 0.08778036, 0.17672077, 0.0234363 , 0.01941025],
       [0.6910351 , 0.08452048, 0.18147002, 0.02350739, 0.01946706],
       [0.6890624 , 0.08359782, 0.18342212, 0.02410621, 0.01981138],
       [0.6652998 , 0.09496945, 0.18707496, 0.02858504, 0.02407077]],
      dtype=float32)

In [41]:
Y_test[:10]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)